In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import random
import os
from torchvision import datasets
import torch.utils.data as data
from sklearn.preprocessing import Normalizer
from sklearn.metrics import f1_score
from PIL import Image
from rdkit import Chem
from rdkit.Chem import AllChem

df_data = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/data/level5_beta_trt_cp_24h_10uM.csv')
pool_drug = set(df_data.SMILES.to_list())

for cell in ['MCF7', 'PC3', 'A549']:
        use_HQ_sample_id = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/revise_use_LINCS_HQ_data_target_cellline_{cell}_sample_id.csv')['x']
        # tmp = pd.read_csv('/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/GPS_predictable_genes.csv')
        # genelist = tmp.x.to_list()
        
        
        df_target = df_data[df_data['sig_id'].isin(use_HQ_sample_id)]
        # median = df_target[['SMILES']+genelist].groupby(by='SMILES').median().reset_index()
        # median['cellline'] = cell
        # df_target = median
        df_target = df_target.rename(columns={'SMILES': 'smiles'})
        # for g in genelist:
        #     df_target[g] = df_target[g].apply(lambda x: (x > 1.5) * 1 + (x >= -1.5) * 1)

        Drugs = df_target['smiles'].to_list()
        tmp = pd.read_csv(f'/egr/research-aidd/menghan1/AnchorDrug/HQ_LINCS_retrain/{cell}_internal_val2_data.csv')
        valDrugs = tmp.SMILES.to_list()
        trainDrugs = set(Drugs)-set(valDrugs)
        print(len(Drugs), len(list(trainDrugs)), len(valDrugs))
        pool_drug = pool_drug.intersection(trainDrugs)
        print(len(list(pool_drug)))

        # df_train = df_target[df_target['smiles'].isin(trainDrugs)]
        # df_val = df_target[df_target['smiles'].isin(valDrugs)]


2084 1203 134
1203
2164 1272 141
563
1562 842 94
288


In [15]:
len(pool_drug)

360